# Lab #4: Curvature

by Peter Richtarik

## The Problem: Ridge Regression

As in Lab 2, in this lab we will consider the ridge regression problem:

$$\min_{x\in \mathbb{R}^n} \left[ f(x) = \frac{1}{2}\|Ax-b\|_2^2 + \frac{\lambda}{2} \|x\|_2^2\right],$$

where $A$ is an $m\times n$ matrix and $b\in \mathbb{R}^n$. Recall that the gradient of $f$ is given by

$$\nabla f(x) =  A^T (Ax-b) + \lambda x$$

and that the Hessian of $f$ is given by

$$\nabla^2 f(x)  = A^T A + \lambda I \overset{\text{def}}{=} H,$$

where $I$ is the  $n\times n$ identity matrix.

It is easy to verify that for all $x,h\in \mathbb{R}^n$ we have

$$ f(x+h) = f(x) + \langle \nabla f(x), h \rangle + \frac{1}{2} \langle H h, h \rangle.$$ 

Hence, $f$ satisfies the 2 assumptions from today's lecture ($G$-strong convexity and $M$-smoothness) with $G=M=H$.

## Goals

In this lab, we shall play with Methods 1, 2 and 3 discussed in today's lecture. Recall that Method 3 (NSync [1]) was covered in Lab 2.

[1] Richtarik and Takac. On optimal probabilities in stochastic coordinate descent methods. Optimization Letters, 2015.

## 1. Data: 3-dimensional problem from lecture

In [3]:
m = 3
n = 3
H = [1.0000 0.9900 0.9999; 0.9900 1.0000 0.9900; 0.9999 0.9900 1.0000]
b = ones(m,1)
lambda = 5
I = eye(3)
l = eigmin(H)
lambda = l / 2
B = H - lambda*I
A = chol(B) # makes sure A is such that H = A'*A + lambda*I 

x_star = (A'*A + lambda*eye(n))\(A'*b) # finds the point x where derivative is zero (the solution)

3x1 Array{Float64,2}:
 -54.3567 
   6.83187
  48.598  

## 2. Ridge regression function

In [44]:
function Ridge(A,b,lambda,x)
    
    g = A*x - b
    return 0.5*(g'*g + lambda*x'*x) 
    
end

Ridge (generic function with 1 method)

## 3. $\tau$-nice sampling

Recall that by sampling we mean a random subset of $[n]=\{1,2,\dots,n\}$. More formally, a sampling  is a random set-valued mapping with values in $2^{[n]}$, where $[n]$. A sampling $\hat{S}$ is uniquely determinied by the probabilities $p_S = \mathbb{P}(\hat{S}=S)$ attached to all subsets $S\subseteq [n]$. 

$\bf Definition.$ $\tau$-nice sampling is the (unique) sampling $\hat{S}$ defined by 

$$\mathbb{P}(\hat{S}=S) = \begin{cases} \frac{1}{n \choose \tau}, & \text{if} \quad |S| = \tau, \\ 0, & \text{otherwise.}\end{cases}$$

In words, this means that $\hat{S}$ is a random subset of $[n] = \{1,2,\dots,n\}$ of cardinality $\tau$, chosen uniformly at random.

### Problem A 
Your task is to implement the $\tau$-nice sampling as a function. In particular, write function $\verb"taunice"$ whose input is $n$ and $\tau\in [n]$ and whose output is a vector $S$ of size $\tau$ such that  $$\hat{S} = \{S[1],\dots,S[\tau]\}.$$

In [34]:
function taunice(n, tau)
    
  ??????
    
  return S
    
end

taunice (generic function with 1 method)

In [37]:
# Testing the taunice function

S = taunice(n,2)
C = H[S,S]  # compute a random submatrix of the Hessian of size 2x2
inv(C)

2x2 Array{Float64,2}:
  50.2513  -49.7487
 -49.7487   50.2513

## 4. Method 1 (Randomized Newton)

The code is written for you. Some pieces are missing.

### Problem B

Fill in the 5 question marks so that the method works as it should.

In [141]:
function Method1(A, b, lambda, tau, T, skip)
    
    println("Method 1 : START")
    
    m, n = size(A)
    
    time = 0
    tic()
    
    H = ???                  # compute the Hessian
    x = zeros(n,1)           # starting point
    g = -b                   # g will be maintained to be equal to A*x - b
  
    time = time + toq()
    
    # remembering and plotting           
    xs = zeros(n, floor(Integer, T/skip) + 1) # will remember the iterates x here
    xs[:,1] = x           # remember iterate x
    println("iteration: $(0), function value: $(Ridge(A,b,lambda,x))")
    
    
    for k = 1:T
        
        tic()
        
        S = ???(n, ???)                          # choose a random index 
        
        d = A[:,S]'*g + lambda*x[S]              # compute partial derivatives for coordinates in S
        update = ???*d                           # compute the update
        x[S] = x[S] -  update                    # perform the update
        g = ???                                  # update the residual
        
        time = time + toq()
        
        # remembering and plotting        
        if k % skip == 0
          xs[:,round(Int,k/skip)+1] = x           # remember iterate x
            println("iteration: $(k), function value: $(Ridge(A,b,lambda,x))")
        end

    end

    println("Time = ", time)
    return x, xs
    
end

Method1 (generic function with 2 methods)

## 5. Method 3 (NSync, copied from Lab 2)

The NSync method  below is copied from Lab 2. It only works with the serial uniform sampling $\hat{S}$. That is, the method updates in each iteration a single coordinate of the vector $x\in \mathbb{R}^n$ chosen uniformly at random. More formally, $\hat{S}$ satsifies 

$$\mathbb{P}(\hat{S}=\{i\}) = \tfrac{1}{n} \qquad \text{for} \qquad i=1,2,\dots,n.$$

### Problem C

In order to compare it with Method 1, implement NSync using the $\tau$-nice sampling. Call the new method $\verb"Method 3"$, and include also an additional input parameter: $\tau$. Be careful to chose the ESO parameters $v_1,\dots,v_n$ inside the function right! Recall they need to satisfy:

$$\mathbf{P} \bullet H \preceq Diag(p \bullet v),$$

where $\mathbf{P}$ is the probability matrix of sampling $\hat{S}$. That is, this is the $n\times n$ matrix with entries $\mathbf{P}_{ij} = \mathbb{P}( \{i,j\} \subset \hat{S})$. 

If $n$ is small, suitable $v$ can be computed through the computation of a maximum eigenvalue. If $n$ is large however, a simpler way for computing suitable $v$ is needed. 

In [23]:
function NSync(A, b, lambda, T)
    
    m, n = size(A)
    
    v = zeros(n,1)
    x = zeros(n,1)    # staring point
    g = -b            # g will be maintained to be equal to A*x - b
    
    for i=1:n         # compute ESO parameters
        v[i] = (norm(A[:,i]))^2 + lambda
    end
    
    for k = 1:T
        i = rand(1:n)                                   # choose a random index 
        h = (1/v[i]) * ( A[:,i]' * g + lambda * x[i] )  # compute the update
        x[i] = x[i] - h[1]                              # perform the update 
        g = g - h[1] * A[:,i]                           # update the residual
    end
    
    return x
    
end

NSync (generic function with 1 method)

In [50]:
function Method3(A, b, lambda, tau, T)
    
    m, n = size(A)
    
    v = zeros(n,1)
    x = zeros(n,1)    # staring point
    g = -b            # g will be maintained to be equal to A*x - b
    
    for i=1:n         # compute ESO parameters
        v[i] = (norm(A[:,i]))^2 + lambda
    end
    
    for k = 1:T
        i = rand(1:n)                                   # choose a random index 
        h = (1/v[i]) * ( A[:,i]' * g + lambda * x[i] )  # compute the update
        x[i] = x[i] - h[1]                              # perform the update 
        g = g - h[1] * A[:,i]                           # update the residual
    end
    
    return x
    
end

Method3 (generic function with 1 method)

## 6. Plotting

This function was not used anywhere in this lab - but feel free to use it.

In [120]:
using PyPlot

function PlotResults(x_star, xs, skip)
    
    ax = axes()
    plt[:plot](skip*(0:length(xs)-1), [sqrt((xs[:,i]-x_star)'*(xs[:,i]-x_star)) for i=1:length(xs)] , ":", linewidth=3.0, label=L"||x - x^*||")
    legend(loc="upper right")
    ylabel("error", fontsize=20)
    xlabel("iterations")
    ax[:set_yscale]("log")
    plt[:show]
    
end

PlotResults (generic function with 2 methods)

## 7. Testing the method(s)

In [145]:
T = 10 # number of iterations
skip = 1

x1, xs = Method1(A,b,lambda,2,T, skip)
println("Error of Method 1 = ", norm(x1-x_star))

x3 = Method3(A,b,lambda,2,T)
println("Error of Method 3 = ", norm(x3-x_star))


# PlotResults(x_star, xs, skip)


Method 1 : START
iteration: 0, function value: [1.5]
iteration: 1, function value: [0.5613023240294428]
iteration: 2, function value: [0.5583477258144952]
iteration: 3, function value: [0.5583477258144952]
iteration: 4, function value: [0.5554227474447875]
iteration: 5, function value: [0.5554227474447876]
iteration: 6, function value: [0.5525270919813898]
iteration: 7, function value: [0.2672834911403561]
iteration: 8, function value: [0.26658102573270065]
iteration: 9, function value: [0.2665792957810137]
iteration: 10, function value: [0.26657929152068593]
Time = 0.000442322
Error of Method 1 = 0.00045807242417239335
Error of Method 3 = 72.92402167626439


## Probem C

Code up also Method 2. Admittedly, this not a practical method. However, code it up anyway and aim to reconstruct the plot for the 3D function from the lecture.

## Problem D

Modify the functions in such a way that you obtain semi-logarithmic plots of function values for all 3 methods.

## Problem E

Rewrite Method 1 so that it solves the logistic regression problem instead.